In [17]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
%run ../SNN.ipynb

In [19]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

In [20]:
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
target_y = T.vector("target Y integer",dtype='int32')

In [21]:
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

conv_1 = lasagne.layers.Conv2DLayer(input_layer, num_filters=8, filter_size=3, name='conv_1', 
                                    nonlinearity=lasagne.nonlinearities.rectify, b=None)

pool_1 = lasagne.layers.Pool2DLayer(conv_1, pool_size=(2,2), name='pool_1', mode='average_inc_pad')

conv_2 = lasagne.layers.Conv2DLayer(pool_1, num_filters=8, filter_size=3, nonlinearity=lasagne.nonlinearities.rectify, 
                                    name='conv_2', b=None)
pool_2 = lasagne.layers.Pool2DLayer(conv_2, pool_size=(2,2), name='pool_2', mode='average_inc_pad')

dense_1 = lasagne.layers.DenseLayer(pool_2, num_units=128, b=None, name='dense_1')

dense_output = lasagne.layers.DenseLayer(dense_1, num_units = 10, nonlinearity = lasagne.nonlinearities.softmax,\
                                         name='output', b=None)

In [22]:
with np.load('cnn_weights_1.npz') as f: 
    param_values = [f['arr_%d' % i] for i in range(len(f.files))] 
    for i in param_values[-1]:
        for j in i:
            j = abs(j)
    for i in param_values:
        for j in i:
            for l, d in enumerate(j):
                if(len(i.shape)>2):
                    for g in d:
                        for k, n in enumerate(g):
                            if(n < 0.000001):
                                g[k] = 0.0
                else:
                    if(d < 0.000001):
                        j[l] = 0.0
lasagne.layers.set_all_param_values(dense_output, param_values)
spiking_net = spiking_from_lasagne(dense_output)

In [23]:
param_values

[array([[[[ 0.        ,  0.        ,  0.07956848],
          [ 0.        ,  0.48179054,  0.20740494],
          [ 0.34697872,  0.79408338,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ,  0.        ],
          [ 0.47595866,  0.30122583,  0.10591751],
          [ 0.9831366 ,  0.41986346,  0.22047783]]],
 
 
        [[[ 0.        ,  0.        ,  0.        ],
          [ 2.04178179,  0.50872241,  0.        ],
          [ 0.15688658,  0.        ,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ,  0.32567081],
          [ 0.        ,  1.90988179,  0.        ],
          [ 0.        ,  0.86825423,  0.        ]]],
 
 
        [[[ 0.        ,  0.14147738,  1.37965923],
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]]],
 
 
        [[[ 0.        ,  0.5231008 ,  0.      

In [32]:
def write_to_file(net, filename='params.txt'):
    info = {'threshold':0.5, 'tau_m':4, 'tau_s':2, 'tau_r':20, 'synapsesPerConnection':1, 'spikesPerSynapse':2,\
            'exitTime':1000,'layersNumber':len(net.layers)}
    f = open(filename, 'w')
    f.writelines([i+': '+str(info[i])+'\n' for i in info])
    
    for l in net.layers:
        if(type(l) == SubSampling2DLayer):
            info = {'PoolLayer':'', 'pool_size': str(" ".join(map(str, l.pool_size)))}
            f.writelines([i+': '+str(info[i])+'\n' for i in info])
        else:
            if(type(l) == InputLayer):
                info = {'InputLayer':'', 'input_shape': str(" ".join(map(str, l.shape)))}
                f.writelines([i+': '+str(info[i])+'\n' for i in info])
            else:
                if(type(l) == Conv2DLayer):
                    params = l.oldweights.tolist()
                    info = {'ConvLayer':'', 'num_filters':l.shape[0], 'filter_size':str(" ".join(map(str, l.filter_shape))), 'weights':''}
                    f.writelines([i+': '+str(info[i])+'\n' for i in info])
                    for i in params:
                        for j in i:
                            for o in j:
                                f.writelines(str(" ".join(map(str, ["%.6f" % k for k in o])))+'\n')
       
                elif(type(l) == DenseLayer):
                    params = l.weights.tolist()
                    info = {'DenseLayer':'', 'num_units:': l.neur_size, 'weights':''}
                    f.writelines([i+': '+str(info[i])+'\n' for i in info])
                    for i in params:
                        f.writelines(str(" ".join(map(str, ["%.6f" % k for k in i])))+'\n')
    f.close()

In [33]:
write_to_file(spiking_net, 'cnn_param.txt')